## Introduction to [pandas](https://pandas.pydata.org/) for connectome tools

- Pandas is a Python library with high-level data structures and data manipulation tools.
- Its most common structure is a DataFrame, a 2d, size-mutable, potentially hetergeneous matrix data structure holding relational data. (Think Excel spreadsheet or SQL table.)
- Essentially, a DataFrame holds data in a tabular fashion, where each column can be a different value type (string, boolean, integer, floating point number, etc.).

For the purposes of this extremely brief tutorial, we will only be working with preexisting DataFrames, so don't worry about the exact syntax below to create a DataFrame:

In [1]:
import pandas as pd  # "pd" is the standard abbrevation for the package

In [24]:
# Don't worry about this syntax for now
students = pd.DataFrame([
    {"name": "Alice", "age": 20, "major": "Math", "school": "Columbia University"},
    {"name": "Bob", "age": 21, "major": "History", "school": "Harvard University"},
    {"name": "Charlie", "age": 22, "major": "Physics", "school": "MIT"},
    {"name": "David", "age": 23, "major": "Computer Science", "school": "Columbia University"},
    {"name": "Eve", "age": 24, "major": "Biology", "school": "University of Washington"},
])

You can view the dataframe in a notebook by simply typing the variable name:

In [4]:
students

,name,age,major,school
0,Alice,20,Math,Columbia University
1,Bob,21,History,Harvard University
2,Charlie,22,Physics,MIT
3,David,23,Computer Science,Columbia University
4,Eve,24,Biology,University of Washington


Alternatively, you can call the `head()` function, which shows the first few rows (optionally passing in the number of rows to show):

In [5]:
students.head()

,name,age,major,school
0,Alice,20,Math,Columbia University
1,Bob,21,History,Harvard University
2,Charlie,22,Physics,MIT
3,David,23,Computer Science,Columbia University
4,Eve,24,Biology,University of Washington


We can also view a particular column with bracket notation, similar to indexing a `dict`:

In [6]:
students["name"]

0      Alice
1        Bob
2    Charlie
3      David
4        Eve
Name: name, dtype: object

You can also get a column by `students.<column_here>`, but this only works when there are no special characters (e.g., spaces, dashes; which may or may not be the case depending on your data)

In [9]:
students.name

0      Alice
1        Bob
2    Charlie
3      David
4        Eve
Name: name, dtype: object

Or a subset of multiple columns:

In [7]:
students[["name", "age"]]

,name,age
0,Alice,20
1,Bob,21
2,Charlie,22
3,David,23
4,Eve,24


### Querying
It is oftentimes useful to subset the dataframe with a particular query. The syntax to subset the rows of a dataframe is: `df[condition]` where `condition` is a boolean vector across the rows of data.

Query example 1: Students at Columbia University.

In [8]:
students[students["school"] == "Columbia University"]

,name,age,major,school
0,Alice,20,Math,Columbia University
3,David,23,Computer Science,Columbia University


Query example 2: Students with an "a" in their name. We'll split this into two steps because it is slightly more complicated. The `apply` method is useful on columns because it will map data. In the first step, we want to lowercase the name:

In [11]:
students["name"].apply(lambda name: name.lower())

0      alice
1        bob
2    charlie
3      david
4        eve
Name: name, dtype: object

Note the apply method takes a single parameter, which is a function mapping the value to the new value. `lambda` functions in python allow us to do this easily in one line with the syntax `lambda param: return_value`. You can also define an external function:

In [12]:
def name_to_lower(name):
    return name.lower()
students["name"].apply(name_to_lower)

0      alice
1        bob
2    charlie
3      david
4        eve
Name: name, dtype: object

Great. Now putting it all together. We want to find students with "a" in the name:

In [13]:
students[students["name"].apply(lambda name: "a" in name.lower())]

,name,age,major,school
0,Alice,20,Math,Columbia University
2,Charlie,22,Physics,MIT
3,David,23,Computer Science,Columbia University


The apply function is very useful in manipulating data. To give a neuroscience example, one might want to find neurons that are a particular distance from some given point. If we have a position in a dataframe, we can `apply` over that position and return whether the distance to some other point is less than some threshold, for example. The `apply` syntax over the entire dataframe is a bit different so here is a brief example:

In [29]:
students[students.apply(lambda row: "a" in row["name"].lower(), axis=1)]

,name,age,major,school
0,Alice,20,Math,Columbia University
2,Charlie,22,Physics,MIT
3,David,23,Computer Science,Columbia University


The `axis` argument tells pandas which axis each mapping spans. It defaults to `axis=0`, which means the lambda function will be applied to entire columns. Since in this example we want to apply to rows of data, we have to include `axis=1`.

Query example 3: (Multiple conditions) Age is odd or studies computer science.

In [14]:
students[(students["age"] % 2 == 1) | (students["major"] == "Computer Science")]

,name,age,major,school
1,Bob,21,History,Harvard University
3,David,23,Computer Science,Columbia University


There are two important things to note here. The first is that we need to wrap the conditions in parentheses. If we don't, we'll get an error:

In [32]:
try:
    # This throws an error!
    students[students["age"] % 2 == 1 | students["major"] == "Computer Science"]
except Exception as e:
    print(e)

Cannot perform 'ror_' with a dtyped [object] array and scalar of type [bool]


The second is the `or` operator is a single pipe `|`. We can also do `and` operations with a single `&`.

### Merging

Oftentimes we have two dataframes and want to merge the data together. Following our student example, suppose some grades are stored elsewhere:

In [35]:
grades = pd.DataFrame([
    {"name": "Alice", "grade": 90},
    {"name": "Beth", "grade": 80},
    {"name": "Charlie", "grade": 85},
    {"name": "David", "grade": 95},
    {"name": "Earl", "grade": 75},
    {"name": "Fiona", "grade": 70},
])

grades

,name,grade
0,Alice,90
1,Beth,80
2,Charlie,85
3,David,95
4,Earl,75
5,Fiona,70


Notice that not every person in `grades` exists in `students`, and vice-versa. This is often the case with data.

How do we combine students with grades? One way to achieve this is with the pandas `merge` function:

In [42]:
pd.merge(
    left = students,
    right = grades,
    on = "name",
    how = "inner"
)
# (The above syntax is broken into lines for clarity but could also be written on one line)

,name,age,major,school,grade
0,Alice,20,Math,Columbia University,90
1,Charlie,22,Physics,MIT,85
2,David,23,Computer Science,Columbia University,95


The syntax will be familiar to anyone who has worked with SQL. In the above example, we perform an `inner` merge with `students` and `grades`. An `inner` merge means the resulting merged dataframe will contain only values that exist in both `students` and `grades`. The `on` specification defines the column that defines which rows to merge. In this case, we want to match students by name.

We can also perform left and right merges which keep data from the left and right dataframes:

In [43]:
pd.merge(
    left = students,
    right = grades,
    on = "name",
    how = "left"
)

,name,age,major,school,grade
0,Alice,20,Math,Columbia University,90.0
1,Bob,21,History,Harvard University,NaN
2,Charlie,22,Physics,MIT,85.0
3,David,23,Computer Science,Columbia University,95.0
4,Eve,24,Biology,University of Washington,NaN


Note that because not all students had a corresponding grade, the grade column in the merged dataframe has some NaN/null values.